In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

In [ ]:
'''
The function below fills in a dictionary with all the data from ball by ball scorecard. A single ball data will be in a list which will have all the
details such as the over that is going on, the ball that is being bowled, the number of runs and what kind of run. The over list also has batsman and bowler
names which will be extracted using the keyword 'to'. The 2 previous elements from 'to' will have the bowler's first name and last name. The 2 elements
ahead of 'to' will have the batsman's first name and last name.
'''
def fill_total_scorecard(total_scorecard,match,batting,bowling,league,over_list):

    total_scorecard["Match"].append(match)
    total_scorecard["Batting Team"].append(batting)
    total_scorecard["Bowling Team"].append(bowling)
    total_scorecard["Overs"].append(over_list[0])
    index = over_list.index('to')
    batsman = over_list[index+1] + ' ' + over_list[index+2]
    bowler = over_list[index-2] + ' ' + over_list[index-1]
    total_scorecard["Batsman"].append(batsman)
    total_scorecard["Bowler"].append(bowler)
    total_scorecard["Runs"].append(over_list[1])
    total_scorecard["Run Type"].append(over_list[-1])
    total_scorecard["Tournament"].append(league)

'''
The function below fills in a dictionary with all the data from batting scorecard. The score_list is basically the information
of each batter in the scorecard. That list contains how many runs the batter made in the match, how many balls the batter played, how many 4s etc.
'''

def fill_batting_scorecard(batting_scorecard,match,batting_side,bowling_side,batsman,league,score_list):

    batting_scorecard["Match"].append(match)
    batting_scorecard["Batting Side"].append(batting_side)
    batting_scorecard["Bowling Side"].append(bowling_side)
    batting_scorecard["Batsman"].append(batsman)
    batting_scorecard["Runs"].append(score_list[0])
    batting_scorecard["Balls"].append(score_list[1])
    batting_scorecard["4s"].append(score_list[2])
    batting_scorecard["6s"].append(score_list[3])
    batting_scorecard["SR"].append(score_list[4])
    batting_scorecard["League"].append(league)

'''
Similar to batting scorecard, the below function fills in the bowling scorecard in a dictionary. The over_list is basically the information
of a bowler's statistics in a match.
'''

def fill_bowling_scorecard(bowling_scorecard,match,batting_side,bowling_side,league,over_list):

    bowling_scorecard["Match"].append(match)
    bowling_scorecard["Batting Side"].append(batting_side)
    bowling_scorecard["Bowling Side"].append(bowling_side)
    bowling_scorecard["Bowler"].append(over_list[0])
    bowling_scorecard["Overs"].append(over_list[1])
    bowling_scorecard["Maidens"].append(over_list[2])
    bowling_scorecard["Runs"].append(over_list[3])
    bowling_scorecard["Wickets"].append(over_list[4])
    bowling_scorecard["Wides"].append(over_list[5])
    bowling_scorecard["No Balls"].append(over_list[6])
    bowling_scorecard["ECON"].append(over_list[7])
    bowling_scorecard["League"].append(league)


'''
The ball by ball data extracted needed some cleaning to make sure all the data in the dictionary is consistent. For example, if there was wicket
taken in a ball, the structure of the list would change. Hence, this will restructure the list filling in consistent information in the correct
places in a list.
If there is no name for a batter or bowler, the list will be filled with 'NA NA' as first and last name.
'''
def restructure_list(over_list):

    if 'Wicket!' in over_list[0]:
        over_list[0]=over_list[0].replace('Wicket!','')
        over_list.insert(1,'W')
        batsman = over_list[-3] + ' ' + over_list[-2]
        over_list[-2] = batsman

    if over_list[0].endswith('0'):
        over_list[0] = over_list[0][:-1]
        over_list.insert(1,'0')

    if over_list.index('to') == 3:
        over_list.insert(3,'NA')
        over_list.insert(3,'NA')

    batsman_index = over_list.index('to') + 1
    if over_list[batsman_index] == ':':
        over_list = over_list[:batsman_index] + ['NA', 'NA'] + over_list[batsman_index+1:]
    if ':' in over_list[batsman_index+1]:
        over_list[batsman_index+1] = over_list[batsman_index+1].replace(':','')

    return over_list

'''
The ball by ball scorecards had different formats. This restructuring of list is for a different format but serves the same purpose. That is
the lists that are being fed to the above function stay consistent. For example, a dot is placed if a there is a dot ball. Instead that dot is
replaced with a '0' run.
Similarly, the run_type will be equal to dismissed and placed in the correct place in the list.
'''
def restructure_list2(over_list):
    over = over_list[0]
    if over_list[1] == '•':
        runs = 0
    else:
        runs = over_list[1]
    if runs == 'W':
        run_type = 'dismissed'
    else:
        run_type = over_list[-1]
    index = over_list.index('to')
    if over_list.index('to') == 2:
        over_list.insert(2,'NA')
    else:
        bowler = over_list[2]
    batsman = over_list[index+1]

    new_over_list = [over,batsman,bowler,runs,run_type]
    return new_over_list

'''
Again, this is for filling the scorecard of ball by ball data but the format of webpage was different. Hence, adjusted it accordingly.
'''

def fill_total_scorecard2(total_scorecard,match,batting,bowling,league,over_list):

    total_scorecard["Match"].append(match)
    total_scorecard["Batting Team"].append(batting)
    total_scorecard["Bowling Team"].append(bowling)
    total_scorecard["Overs"].append(over_list[0])
    total_scorecard["Batsman"].append(over_list[1])
    total_scorecard["Bowler"].append(over_list[2])
    total_scorecard["Runs"].append(over_list[3])
    total_scorecard["Run Type"].append(over_list[-1])
    total_scorecard["Tournament"].append(league)

'''
3 arguments are given. This is to ensure that the team names being using are complete as multiple teams have same name but additional text added
such as 'Durham City CC Blue' or 'Durham City CC Gold'.
'''
def extract_match(team_names,team_names2,playing_teams):
    team_name1 = team_names[0].text
    team_name2 = team_names[1].text
    complete_team_name1 = team_name1 + ' ' + team_names2[0].text.replace('\n','').strip()
    complete_team_name2 = team_name2 + ' ' + team_names2[1].text.replace('\n','').strip()
    match = complete_team_name1 + ' vs ' + complete_team_name2
    batting_team = playing_teams[0]
    bowling_team = playing_teams[-1].strip()
    if batting_team in complete_team_name1:
        batting_team = complete_team_name1
        bowling_team = complete_team_name2
    else:
        batting_team = complete_team_name2
        bowling_team = complete_team_name1
    return match,batting_team,bowling_team

'''
This is used to check whether the match has been already recorded or not. This will prevent us from duplication of data.
'''
def check_match(matches_played,match):
    for matches in matches_played:
        if matches == match:
            return True
        else:
            return False

In [ ]:
LEAGUE = 'U-13 League'
SEASON = '2023'


In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get('https://dcbjuniorlge.play-cricket.com/home')

time.sleep(3)


In [ ]:
driver.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()

#Goes to the sign-in page and enters the details

driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/ul/li[3]/a').click()
time.sleep(3)

email_box = driver.find_element(By.XPATH, '//*[@id="email"]')
USERNAME = input('Enter your username: ')
email_box.send_keys(USERNAME)
password_box = driver.find_element(By.XPATH, '//*[@id="password"]')
password_box.click()
time.sleep(5)
PASSWORD = input('Enter your password: ')
password_box.send_keys(PASSWORD)
time.sleep(3)
driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div[1]/div/div/form/div[3]/button[1]').click()
time.sleep(10)


In [ ]:
#Select the season to extract all the leagues
select = Select(driver.find_element(By.XPATH,'//*[@id="season"]'))
select.select_by_visible_text(SEASON)
time.sleep(3)



In [ ]:
#Gather all the league information and their related links
soup = BeautifulSoup(driver.page_source, 'html.parser')
all_competitions = soup.find_all('div', class_='col-sm-4')
league_information = {'League':[],
                      'Link': []}
for competition in all_competitions:

    all_tournaments = competition.find_all('li')
    for tourna in all_tournaments:
        if LEAGUE in tourna.text:
            league_href = tourna.find('a').get('href')
            league_link = 'https://dcbjuniorlge.play-cricket.com/'+league_href
            league_information['League'].append(tourna.text)
            league_information['Link'].append(league_link)


In [ ]:
#Navigating through the league to find all the relevant team links in the league
team_information = {"Team Name": [],
                   "Team Link": []}
for info in league_information['Link']:
    driver.get(info)

    league_soup = BeautifulSoup(driver.page_source, 'html.parser')
    teams_link = league_soup.find_all('td', class_='text-left team-name')
    for team in teams_link:
        team_information["Team Name"].append(team.text)
        team_information["Team Link"].append(team.find('a').get('href'))


In [ ]:
team_information["Scorecard Link"] = []
team_number = 0
#Looping through all the teams in the list
for link in team_information["Team Link"]:
    driver.get(link)
    time.sleep(2)
    group = driver.find_element(By.XPATH, '/html/body/div[12]/div[2]/div[2]/div/h3').text
    team = driver.find_element(By.XPATH, '/html/body/div[12]/div[2]/div[2]/div/h5').text
    team_information["Team Name"][team_number] = f'{group} {team}'
    #Open the relevant season
    try:
        driver.find_element(By.LINK_TEXT, SEASON).click()
    except:
        team_information["Scorecard Link"].append("No 2023")
    else:
        time.sleep(2)

        #Extract all the matches url
        matches_url = driver.current_url
        team_url = matches_url.split('/')[2]

        matches_soup = BeautifulSoup(driver.page_source,'html.parser')
        scorecard_tag = matches_soup.find_all('a', class_='link-scorecard d-none d-md-inline-block rounded-circle')
        team_information["Scorecard Link"].append([])
        for scorecard in scorecard_tag:
            scorecard_href = scorecard.get('href')
            team_information["Scorecard Link"][team_number].append(f'https://{team_url}{scorecard_href}')

    team_number += 1


In [ ]:
#Storing the link of all the scorecards for those teams who have played in 2023 season.
team_information['Scorecard Link'] = [item for item in team_information['Scorecard Link'] if item != 'No 2023']

In [ ]:
total_scorecard = {"Match": [],
                   "Batting Team": [],
                   "Bowling Team": [],
                   "Overs": [],
                   "Batsman": [],
                   "Bowler": [],
                   "Runs": [],
                   "Run Type": [],
                   "Tournament": [],
                   }

batting_scorecard = {'Match': [],
                    'Batting Side': [],
                    'Bowling Side': [],
                    'Batsman': [],
                    'Runs': [],
                    'Balls': [],
                    '4s': [],
                    '6s': [],
                    'SR': [],
                    'League': []}

bowling_scorecard = {'Match': [],
                    'Batting Side': [],
                    'Bowling Side': [],
                    'Bowler': [],
                    'Overs': [],
                    'Maidens': [],
                    'Runs': [],
                    'Wickets': [],
                    'Wides': [],
                    'No Balls': [],
                    'ECON': [],
                    'League': []}

matches_played = []

In [ ]:
'''
In case if there is an error in scraping the data, then uncomment the commented and comment the start_found = True and run this cell.
'''
# error_link = 'https://annfieldplain.play-cricket.com/website/results/5832775'
#start_found = False
start_found = True

In [ ]:
'''
The scorecard scrapping starts here. The code will go through all the links one by one and extract data. There are multiple try and except methods below
this is due to different format of webpages for different teams.
'''
for all_links in team_information["Scorecard Link"]:
    for link in all_links:
        if start_found or error_link == link:
            start_found = True
            current_link = driver.get(link)
            time.sleep(5)
            current_soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                #This line finds the name of the current league in case you are trying to find the scorecard of the specific league
                current_league = current_soup.find_all('span', class_='d-block d-md-inline')[1].text
            except:
                print('Tournament not required')
            else:
'''
This line will extract the status of the match. If the match is not abandoned then it will look into the scorecards of the match or else it will pass
onto the next scorecard link
'''
                match_status = current_soup.find('div', class_='match-status-row').text.strip().lower()
                if LEAGUE.lower() in current_league.lower() and ('won' in match_status or 'tied' in match_status):
                    team_names = current_soup.find_all('p', class_='team-name')
                    team_names2 = current_soup.find_all('span', class_='team-info-1')
                    complete_team_name1 = team_names[0].text + ' ' + team_names2[0].text.replace('\n','').strip()
                    complete_team_name2 = team_names[1].text + ' ' + team_names2[1].text.replace('\n','').strip()
                    match = complete_team_name1 + ' vs ' + complete_team_name2
                    date = current_soup.find('div',class_='col-sm-12 col-md-6 col-lg-6 text-lg-right leaguedetail-right').text.strip().split()
                    match_detail = f'{match} {date[0]} {date[1]} {date[2]}'
                    if match_detail in matches_played:
                        print(f'{match} has already been recorded')
                    else:
                        playing_teams = []
                        try:
                            driver.find_element(By.XPATH, '//*[@id="iasScorecardtab-tab"]').click()
                            time.sleep(5)
                        except:
                            try:
                                driver.find_element(By.XPATH,'//*[@id="nvScorecardTab-tab"]').click()
                                time.sleep(5)
                            except:
                                playing_teams = driver.find_element(By.XPATH, '//*[@id="myTab"]').text.split('\n')
                                all_names = extract_match(team_names, team_names2,playing_teams)
                                batting_team = all_names[1]
                                bowling_team = all_names[2]
                                print('Match: ' + match + '\n')
                                #Extracting the simple Batting Scorecard
                                batting_table = current_soup.find_all('table', class_='table standm table-hover')
                                for table in batting_table:
                                    all_table = table.find_all('tr')[1:]
                                    for row in all_table:
                                        batsman = row.find('div', class_='bts').text
                                        all_data = row.find_all('td')[1:]
                                        row_data = [data.text.strip() for data in all_data][2:]
                                        fill_batting_scorecard(batting_scorecard,match,batting_team,bowling_team,batsman,current_league,row_data)
                                    temp_variable = batting_team
                                    batting_team = bowling_team
                                    bowling_team = temp_variable

                                #Extracting the simple Bowling Scorecard here
                                bowling_table = current_soup.find_all('table', class_='table bowler-detail table-hover')
                                for table in bowling_table:
                                    all_table = table.find_all('tr')[1:]
                                    for row in all_table:
                                        all_data = row.find_all('td')
                                        row_data = [data.text.strip() for data in all_data]
                                        fill_bowling_scorecard(bowling_scorecard,match,batting_team,bowling_team,current_league,row_data)
                                    temp_variable = batting_team
                                    batting_team = bowling_team
                                    bowling_team = temp_variable
                            else:
                                current_soup = BeautifulSoup(driver.page_source,'html.parser')
                                playing_teams.append(current_soup.find_all('div',class_='nvp-innings__tab-team')[0].text)
                                playing_teams.append(current_soup.find_all('div',class_='nvp-innings__tab-team')[1].text)
                                all_names = extract_match(team_names, team_names2, playing_teams)
                                batting_team = all_names[1]
                                bowling_team = all_names[2]
                                print('Match: ' + match + '\n')
                                #Extracting both Batting and Bowling Scorecard (a different webpage format)

                                first_team = current_soup.find_all('div', class_='nv-g-flex nvp-scorecard__table-row')[:11]
                                for player in first_team:
                                    player_list = player.text.split()
                                    row_data = []
                                    batsman = player_list[0] + ' ' + player_list[1]
                                    try:
                                        row_data.extend([player_list[-6],player_list[-5],player_list[-3],player_list[-2],player_list[-1]])
                                    except:
                                        #In case the batter hasn't played the match, all its statistics will have NA
                                        row_data.extend(['NA','NA','NA','NA','NA'])

                                    fill_batting_scorecard(batting_scorecard,match,batting_team,bowling_team,batsman,current_league,row_data)

                                i = 2
                                last_player = True
                                while(last_player):
                                    try:
                                        bowler = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[1]').text
                                        over = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[2]').text
                                        maiden = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[3]').text
                                        runs = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[4]').text
                                        wickets = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[5]').text
                                        wides = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[6]').text
                                        no_balls = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[7]').text
                                        econ = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[1]/div[3]/div[{i}]/div[8]').text
                                    except:
                                        last_player = False
                                    else:
                                        row_data = [bowler,over,maiden,runs,wickets,wides,no_balls,econ]
                                        fill_bowling_scorecard(bowling_scorecard,match,batting_team,bowling_team,current_league,row_data)
                                        i += 1
                                temp_variable = batting_team
                                batting_team = bowling_team
                                bowling_team = temp_variable

                                #Click on the second innings
                                driver.find_element(By.XPATH,'//*[@id="ancInnings2"]').click()
                                second_team = current_soup.find_all('div', class_='nv-g-flex nvp-scorecard__table-row')[11:]
                                for player in second_team:
                                    player_list = player.text.split()
                                    row_data = []
                                    batsman = player_list[0] + ' ' + player_list[1]
                                    try:
                                        row_data.extend([player_list[-6],player_list[-5],player_list[-3],player_list[-2],player_list[-1]])
                                    except:
                                        row_data.extend(['NA','NA','NA','NA','NA'])

                                    fill_batting_scorecard(batting_scorecard,match,batting_team,bowling_team,batsman,current_league,row_data)

                                i = 2
                                last_player = True
                                #All the players are in a column, it will run until last player's statistics are extracted
                                while(last_player):
                                    try:
                                        bowler = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[1]').text
                                        over = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[2]').text
                                        maiden = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[3]').text
                                        runs = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[4]').text
                                        wickets = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[5]').text
                                        wides = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[6]').text
                                        no_balls = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[7]').text
                                        econ = driver.find_element(By.XPATH,f'//*[@id="nvp-widget-3"]/div/div/div/div/div[2]/div[3]/div[{i}]/div[8]').text
                                    except:
                                        last_player = False
                                    else:
                                        row_data = [bowler,over,maiden,runs,wickets,wides,no_balls,econ]
                                        fill_bowling_scorecard(bowling_scorecard,match,batting_team,bowling_team,current_league,row_data)
                                        i += 1
                                temp_variable = batting_team
                                batting_team = bowling_team
                                bowling_team = temp_variable


                        else:
                            #The third webpage format
                            time.sleep(5)
                            current_soup = BeautifulSoup(driver.page_source, 'html.parser')
                            try:
                                playing_teams.append(driver.find_element(By.XPATH, '//*[@id="iasScorecard"]/div/div[1]/ul/li[1]').text)
                                playing_teams.append(driver.find_element(By.XPATH, '//*[@id="iasScorecard"]/div/div[1]/ul/li[2]').text)
                            except:
                                print('No Scorecard Information')
                            else:
                                all_names = extract_match(team_names, team_names2,playing_teams)
                                batting_team = all_names[1]
                                bowling_team = all_names[2]
                                print('Match: ' + match + '\n')
                                #Batting Scorecard
                                for n in range(2):
                                    no_of_players = 2
                                    current_soup = BeautifulSoup(driver.page_source, 'html.parser')
                                    all_batters = current_soup.find('div', class_='SmartTableStyle__TableContainer-sc-1m7nz3v-0 jDknGc').find_all('span')
                                    for batter in all_batters:
                                        if not batter.text:
                                            pass
                                        else:
                                            no_of_players +=1

                                    for i in range(2,no_of_players):
                                        row_data = []
                                        batsman = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[1]/div/div[1]').text
                                        runs = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[2]/div').text
                                        balls = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[3]').text
                                        fours = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[4]').text
                                        sixes = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[5]').text
                                        sr = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[2]/div/div[{i}]/div[6]').text

                                        if not runs or not balls or not fours or not sixes or not sr:
                                            row_data = ["NA"]*6
                                        else:
                                            row_data.append(runs)
                                            row_data.append(balls)
                                            row_data.append(fours)
                                            row_data.append(sixes)
                                            row_data.append(sr)

                                        fill_batting_scorecard(batting_scorecard,match,batting_team,bowling_team,batsman,current_league,row_data)


                                    all_bowlers = current_soup.find_all('div',class_='SmartTableStyle__TableContainer-sc-1m7nz3v-0 jDknGc')[1].find_all('span')
                                    no_of_bowlers = 0
                                    for bowler in all_bowlers:
                                        if not bowler.text:
                                            pass
                                        else:
                                            no_of_bowlers +=1
                                    j=2
                                    for z in range(no_of_bowlers):
                                        row_data = []
                                        bowler = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[1]').text
                                        overs = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[2]').text
                                        maiden = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[3]').text
                                        runs = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[4]').text
                                        wickets = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[5]').text
                                        wides = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[6]').text
                                        no_balls = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[7]').text
                                        econ = driver.find_element(By.XPATH,f'//*[@id="iasScorecard"]/div/div[4]/div/div[{j}]/div[8]').text

                                        row_data.extend([bowler, overs, maiden, runs, wickets, wides, no_balls, econ])
                                        fill_bowling_scorecard(bowling_scorecard,match,batting_team,bowling_team,current_league,row_data)
                                        j += 1


                                    temp_variable = batting_team
                                    batting_team = bowling_team
                                    bowling_team = temp_variable
                                    try:
                                        driver.find_element(By.XPATH,'//*[@id="iasScorecard"]/div/div[1]/ul/li[2]').click()

                                    except:
                                        driver.find_element(By.XPATH,'//*[@id="iasScorecard"]/div/div[1]/ul/li[1]').click()

                                    time.sleep(5)
                                    current_soup = BeautifulSoup(driver.page_source, 'html.parser')

                        #Ball By Ball

                        driver.find_element(By.LINK_TEXT, 'BALL BY BALL').click()
                        time.sleep(5)
                        for y in range(1,3):
                            try:
                                driver.find_element(By.XPATH,f'//*[@id="iasBallbyball"]/div/div[1]/ul/li[{y}]').click()
                                time.sleep(5)
                            except:
                                try:
                                    driver.find_element(By.XPATH,f'//*[@id="ancInnings{y}"]').click()
                                    time.sleep(5)
                                except:
                                    try:
                                        prev_height = -1
                                        max_scrolls = 100
                                        scroll_count = 0
                                        while scroll_count < max_scrolls:
                                            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                                            time.sleep(3)
                                            new_height = driver.execute_script("return document.body.scrollHeight")
                                            if new_height == prev_height:
                                                break
                                            prev_height = new_height
                                            scroll_count += 1
                                        all_overs = current_soup.find_all('li',class_='nvp-commentary__balls_list__ball')
                                        for over in all_overs:
                                            over_list = restructure_list2(over.text.split())
                                            fill_total_scorecard2(total_scorecard,match,batting_team,bowling_team,current_league,over_list)
                                        temp_variable = batting_team
                                        batting_team = bowling_team
                                        bowling_team = temp_variable
                                    except:
                                        print('Game was not digitally recorded')
                                        matches_played.append(match)
                                else:
                                    prev_height = -1
                                    max_scrolls = 100
                                    scroll_count = 0
                                    while scroll_count < max_scrolls:
                                        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                                        time.sleep(3)
                                        new_height = driver.execute_script("return document.body.scrollHeight")
                                        if new_height == prev_height:
                                            break
                                        prev_height = new_height
                                        scroll_count += 1
                                    all_overs = current_soup.find_all('li',class_='nvp-commentary__balls_list__ball')
                                    for over in all_overs:
                                        over_list = restructure_list2(over.text.split())
                                        fill_total_scorecard2(total_scorecard,match,batting_team,bowling_team,current_league,over_list)
                                    temp_variable = batting_team
                                    batting_team = bowling_team
                                    bowling_team = temp_variable


                            else:
                                current_soup = BeautifulSoup(driver.page_source, 'html.parser')
                                all_overs = current_soup.find_all('div', class_='BallDetailViewStyle__BallContainer-sc-19vwj2-1')
                                for over in all_overs:
                                    over_list = restructure_list(over.text.split())
                                    fill_total_scorecard(total_scorecard,match,batting_team,bowling_team,current_league,over_list)

                                temp_variable = batting_team
                                batting_team = bowling_team
                                bowling_team = temp_variable

                        matches_played.append(match_detail)











Match: Lanchester CC Under 13 vs Annfield Plain CC Under 13

Match: Lanchester CC Under 13 vs Consett CC Under 13

Match: Lanchester CC Under 13 vs Burnhope CC Under 13

Lanchester CC Under 13 vs Leadgate CC Under 13 has already been recorded
No Scorecard Information
Match: Consett CC Under 13 vs Lanchester CC Under 13

Match: Lanchester CC Under 13 vs Blaydon CC Under 13

Match: Burnhope CC Under 13 vs Lanchester CC Under 13

Leadgate CC Under 13 vs Lanchester CC Under 13 has already been recorded
Lanchester CC Under 13 vs Annfield Plain CC Under 13 has already been recorded
Lanchester CC Under 13 vs Consett CC Under 13 has already been recorded
Match: Consett CC Under 13 vs Beamish and E. Stanley CC Under 13

Leadgate CC Under 13 vs Consett CC Under 13 has already been recorded
Consett CC Under 13 vs Annfield Plain CC Under 13 has already been recorded
Match: Consett CC Under 13 vs Blaydon CC Under 13

Consett CC Under 13 vs Lanchester CC Under 13 has already been recorded
Match: Con

In [ ]:
total_scorecard_df = pd.DataFrame(total_scorecard)
total_scorecard_df.to_csv('Total Scorecard (Ball By Ball).csv', index=False)

In [ ]:
batting_scorecard_df = pd.DataFrame(batting_scorecard)
batting_scorecard_df.to_csv('Batting Scorecard.csv', index=False)

In [ ]:
bowling_scorecard_df = pd.DataFrame(bowling_scorecard)
bowling_scorecard_df.to_csv('Bowling Scorecard.csv', index=False)